### Proyecto 1B - Mapping Data Flow para carga de archivos JSON desde ADLS hacia Azure SQL DB

#### Paso 8 - Creación de un Linked Service que haga referencia a Azure SQL DB

28. Creación y configuración del Linked Service que hace referencia hacia Azure SQL DB

<center><img src="https://i.postimg.cc/Yqpjr13w/adf305.png"></center>
<center><img src="https://i.postimg.cc/85RPD5f5/adf306.png"></center>

Indicamos el **Server name**, la **base de datos**, el **user name** y la **password**.
Nos conectaremos sin utilizar la seguridad por medio de secretos que nos brinda Azure Key Vault

<center><img src="https://i.postimg.cc/qMbBpWq8/adf307.png"></center>
<center><img src="https://i.postimg.cc/g0RGqJDD/adf308.png"></center>


#### Paso 9 - Creación de los datasets

29. Creación y configuración de los dataset de origen y destino

    El dataset de origen será **ds_Valid_IPL_Data_Json**. Este archivo tendrá como referencia a un único archivo, 
    al archivo **ipl 2008.json**. Estamos referenciando a un solo archivo por motivos de prueba, si vemos que se
    carga sin problemas, mas adelante cargaremos todos los archivos JSON del contenedor **raw**

<center><img src="https://i.postimg.cc/pL0tfjzz/adf309.png"></center>
<center><img src="https://i.postimg.cc/TwbMvHSJ/adf310.png"></center>
<center><img src="https://i.postimg.cc/WzPRNwcL/adf311.png"></center>
<center><img src="https://i.postimg.cc/5y5h6QfH/adf312.png"></center>
<center><img src="https://i.postimg.cc/Ghd06Kjv/adf313.png"></center>

El dataset de destino será **ds_AzureSqlTable_IPL_Data**. Este dataset hace referencia a la tabla **dbo.tbl_IPLData**

<center><img src="https://i.postimg.cc/RFSfjjqj/adf314.png"></center>
<center><img src="https://i.postimg.cc/QCV7M1Qm/adf315.png"></center>
<center><img src="https://i.postimg.cc/nzhjdBPx/adf316.png"></center>


#### Paso 10 - Creación de un nuevo Pipeline

30. Creación y configuración de un nuevo Pipeline que se llamará **pl_Copy_Valid_IPL_To_SQL** que nos permitirá 
    realizar la carga del archivo **ipl 2008.json** en la tabla **dbo.tbl_IPLData**

    Agregaremos la actividad **Copy Data**

<center><img src="https://i.postimg.cc/MTt9f7hj/adf317.png"></center>
<center><img src="https://i.postimg.cc/j2FvZsqx/adf318.png"></center>
<center><img src="https://i.postimg.cc/pXmsynkp/adf319.png"></center>    
<center><img src="https://i.postimg.cc/xd25spQz/adf320.png"></center>

Esta es la estructura de la tabla **dbo.tbl_IPLData**

<center><img src="https://i.postimg.cc/G29yzv4D/adf321.png"></center>

Esta es la estructura que tiene el archivo **ipl 2008.csv**, pero lo vemos como ejemplo, para ver los nombres de
las columnas

<center><img src="https://i.postimg.cc/5t2pz7Mq/adf322.png"></center>

Realizamos el mapeo de las columnas del archivo **ipl 2008.json** frente a las columnas de la tabla **dbo.tbl_IPLData**

<center><img src="https://i.postimg.cc/gcZs3L15/adf323.png"></center>

Eliminamos la columna **Date**

<center><img src="https://i.postimg.cc/bN0RBcy9/adf324.png"></center>

31. Vamos a **Validar** y luego ejecutar el pipeline utilizando **Debug**.

<center><img src="https://i.postimg.cc/Vs0wM4f8/adf326.png"></center>
<center><img src="https://i.postimg.cc/VvQVXqgf/adf327.png"></center>

Podemos ver que en total se cargaron 59 registros y la columna **Date** de la tabla solo tiene valores NULL

<center><img src="https://i.postimg.cc/MH71NVSp/adf325.png"></center>

#### Paso 11 - Cargar todos los archivos JSON del contenedor "raw" de nuestro ADLS utilizando Data Flows

32. Para ello previamente vamos a truncar la tabla **dbo.tbl_IPLData**:

    TRUNCATE TABLE [dbo].[tbl_IPLData]

<center><img src="https://i.postimg.cc/VL3dndQ7/adf328.png"></center>

33. Crearemos un Data Flow llamado **df_Copy_IPL_Data**

<center><img src="https://i.postimg.cc/Hn83vVSS/adf329.png"></center>

En el dataset de origen **ds_Valid_IPL_Data_Json** dejaremos vacio el recuadro del archivo, dado que utilizaremos
un comodin para llamar a todos los archivos JSON

<center><img src="https://i.postimg.cc/c4wGrHsM/adf330.png"></center>    

En la opción **Column to store file name** debemos colocar **SourceFileName**. De esta manera, estaremos agregando
una nueva columna con el nombre del archivo al cual cada registro pertenece

<center><img src="https://i.postimg.cc/rwTXWks6/adf331.png"></center>
<center><img src="https://i.postimg.cc/W17RwHqd/adf332.png"></center>    

Podemos visualizar en el **Data Preview** que vamos a tener columnas con valores NULL, por tanto, tenemos que
modificar el tipo de datos para aquellas columnas

<center><img src="https://i.postimg.cc/KYc6qDtJ/adf333.png"></center>
<center><img src="https://i.postimg.cc/85jqDkKv/adf334.png"></center>    
<center><img src="https://i.postimg.cc/k4zL5rzp/adf335.png"></center>

Crearemos una tarea **sink** que haga referencia hacia Azure SQL DB, a nuestro dataset de destino **ds_AzureSqlTable_IPL_Data**

<center><img src="https://i.postimg.cc/DfQtTN5B/adf336.png"></center>
<center><img src="https://i.postimg.cc/N0rzPC58/adf337.png"></center>    
<center><img src="https://i.postimg.cc/NFQCFgbb/adf338.png"></center>

Debemos mapear las columnas obtenidas del **Source** con las columnas de la tabla 

<center><img src="https://i.postimg.cc/vHq28wnL/adf339.png"></center>  
<center><img src="https://i.postimg.cc/05ZcZ1VR/adf341.png"></center>

34. Creamos un nuevo Pipeline llamado **pl_Copy_Full_IPL_Data_using_DF** en el cual ejecutaremos nuestro Data Flow. Vamos a 
    **Validar** y luego ejecutar el pipeline utilizando **Debug**.

<center><img src="https://i.postimg.cc/52zBhhSt/adf343.png"></center>   

Podemos ver que en total se cargaron 898 registros y la columna **Date** de la tabla solo tiene valores NULL.
Debemos corregir este error que tenemos con la columna **Date**

<center><img src="https://i.postimg.cc/YSf19QVK/adf344.png"></center>
<center><img src="https://i.postimg.cc/XND9pj2J/adf345.png"></center>

#### Paso 12 - Corrección problema columna "Date"

32. Vamos a corregir el problema de tipo de dato que tenemos con la columna **Date**

    Agregamos una actividad **Derived Column** intermedia para generar una nueva columna llamada **NewDate** que tendrá 
    los datos de la columna **Date** formateados

<center><img src="https://i.postimg.cc/KvRwMYxs/adf346.png"></center>

Vamos a utilizar la expresión: **coalesce(toDate(Date,'dd-MM-yyyy'))**

<center><img src="https://i.postimg.cc/T2rqHMQY/adf347.png"></center>

En la tarea **sink** debemos mapear la nueva columna **NewDate** que acabamos de crear con la columna **Date** de la tabla

<center><img src="https://i.postimg.cc/pLyjvgQj/adf348.png"></center>

33. Previo a ejecutar nuestro Pipeline, truncaremos la tabla **dbo.tbl_IPLData**:

    TRUNCATE TABLE [dbo].[tbl_IPLData] 

34. Vamos a **Validar** y luego ejecutar el pipeline utilizando **Debug**.

<center><img src="https://i.postimg.cc/5tL6N1XX/adf349.png"></center>

35. Ahora podemos ver que nuestra columna se visualiza de manera correcta

<center><img src="https://i.postimg.cc/QtDVFP70/adf350.png"></center>



